In [ ]:
import os
import numpy as np
import math
from math import pi, ceil, sqrt, pow
import time
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
part_rad = 1.0 # Radius of interior particle

In [ ]:
'''Read the initial state of particles'''
'''Calculating the particle counts'''
df0 = pd.read_csv("ini_state_q1.txt", header = None, delimiter='\t', names=['x','y','r','theta','speed','type'])
part = df0['type']
x = df0['x']
y = df0['y']
N_ACTIVE = np.sum([part==1])
N_PASSIVE = np.sum([part==-1])
N_WALL = np.sum([part==0])
PASSIVE_HALF = int(N_PASSIVE/2)
N_INT = int(N_ACTIVE + N_PASSIVE)
N_TOTAL = int(N_ACTIVE + N_PASSIVE + N_WALL)

In [ ]:
cell_width = 4*part_rad
nn_cell_width = 4*part_rad
sq_cutoff_rad = (nn_cell_width)**2 # square of cut-off radius

In [ ]:
x_low = np.amin(x)-nn_cell_width
y_low = np.amin(y)-nn_cell_width
x_max = np.amax(x)+nn_cell_width
y_max = np.amax(y)+nn_cell_width

Nc_x = int(np.ceil((x_max-x_low)/nn_cell_width))
Nc_y = int(np.ceil((y_max-y_low)/nn_cell_width))
N_cells = int(Nc_x*Nc_y) # Total no of cells

Calculating the Mixing Index using Nearest Neighbour in a cutoff radius

In [ ]:
def nn_fn(data_passive):
    nn_list = np.zeros(N_PASSIVE,dtype=int)
    posx = data_passive[:,0]
    posy = data_passive[:,1]

    head = np.ones(N_cells)*(-1)
    cell_x = (np.divide(np.subtract(posx,x_low),nn_cell_width)).astype(int) # Putting agent into respective xcell
    cell_y = (np.divide(np.subtract(posy,y_low),nn_cell_width)).astype(int); # Putting agent into respective ycell
    cell_pos = (cell_x*Nc_y + cell_y).astype(int); # Cell number calculated vertically
    
    #assigning particles' (identity) in head2 & nn_list for easily accessing
    for i in range (N_PASSIVE):
        nn_list[i] = head[cell_pos[i]]
        head[cell_pos[i]] = i
        
    pass_type = np.zeros((N_PASSIVE,2))

    for i in range (Nc_x):
        for j in range (Nc_y):
            cell_n = i*Nc_y + j
            iat = int(head[cell_n])
            while (iat != -1): # -1 implies no particles in this head/cell 
                for neigh_cell_x in [i-1,i,i+1]:
                    for neigh_cell_y in [j-1,j,j+1]:
                        neigh_cell_n = neigh_cell_x*Nc_y + neigh_cell_y
                        jat = int(head[neigh_cell_n])
                        while (jat != -1): # -1 implies no particles in this head/cell 
                            if(iat != jat):
                                # Distance calculation of neighboring particles
                                dx = posx[jat] - posx[iat]
                                dy = posy[jat] - posy[iat]
                                sq_distance = dx*dx + dy*dy
                                
                                if (sq_distance <=sq_cutoff_rad):
                                    if(jat<PASSIVE_HALF):
                                        pass_type[iat][0] += 1 # TYPE 1 PASSIVE
                                    if(jat>=PASSIVE_HALF):
                                        pass_type[iat][1] += 1 # TYPE 2 PASSIVE

                            jat = int(nn_list[jat])

                iat = nn_list[iat]
    #print(np.shape(pass_type))
    ratio_1 = np.sum(2*np.nan_to_num(np.divide(pass_type[:PASSIVE_HALF,1],(pass_type[:PASSIVE_HALF,0]+pass_type[:PASSIVE_HALF,1]))),axis=0)
    ratio_2 = np.sum(2*np.nan_to_num(np.divide(pass_type[-PASSIVE_HALF:,0],(pass_type[-PASSIVE_HALF:,0]+pass_type[-PASSIVE_HALF:,1]))),axis=0)
    mix_index = (ratio_1+ratio_2)/N_PASSIVE
#     print(pass_type)
#     print(mix_index)
    return mix_index

In [ ]:
ini_time = 0
max_time = 50000

In [ ]:
time = np.arange(ini_time,max_time+1,20).reshape(-1,1)
steps = np.shape(time)[0]

In [ ]:
df0 = pd.read_csv("ini_state_q1.txt", header = None, delimiter='\t', names=['x','y','r','theta','speed','type'])
state0 = df0.to_numpy(dtype=float)
data0 = state0[N_ACTIVE:N_INT,:2]

In [ ]:
avg_rew_20 = np.zeros((steps,1),dtype=float)

In [ ]:
avg_rew_20[0,0] = nn_fn(data0) 
df = pd.read_csv("MI_vs_Time_opt.csv", header=None, delimiter='\t')
state = df.to_numpy(dtype=float)
sim_time = state[:,0]
sim_rew = state[:,1:]
avg_rew_20[1:np.shape(sim_time)[0]+1,0] = np.sum(sim_rew, axis=1)/np.shape(sim_rew)[1]
del df, state, sim_time, sim_rew

# With std dev

In [ ]:
df = pd.read_csv("MI_vs_Time_opt.csv", header=None, delimiter='\t')
state = df.to_numpy(dtype=float)
sim_time = state[:,0]
sim_rew = np.zeros((np.shape(sim_time)[0]+1,np.shape(state)[1]-1),dtype=float)
sim_rew[0,:] = nn_fn(data0)
sim_rew[1:,:] = state[:,1:]
mean_rew_l = avg_rew_20[:,0].reshape(-1,1)

In [ ]:
std=np.zeros((np.shape(sim_time)[0]+1,1),dtype=float)

In [ ]:
std = np.sqrt(np.sum(np.square(sim_rew-mean_rew_l),axis=1)/(np.shape(sim_rew)[1]-1)).reshape(-1,1)

In [ ]:
y_plus_l = (mean_rew_l+std).flatten()
y_minus_l = (mean_rew_l-std).flatten()
x = np.arange(0,50001,20)
del sim_time, sim_rew, std, avg_rew_20

In [ ]:
avg_rew_20 = np.zeros((steps,1),dtype=float)

In [ ]:
avg_rew_20[0,0] = nn_fn(data0) 
df = pd.read_csv(r"MI_vs_Time_non_opt.csv", header=None, delimiter='\t')
state = df.to_numpy(dtype=float)
sim_time = state[:,0]
sim_rew = state[:,1:]
avg_rew_20[1:np.shape(sim_time)[0]+1,0] = np.sum(sim_rew, axis=1)/np.shape(sim_rew)[1]
del df, state, sim_time, sim_rew

In [ ]:
df = pd.read_csv(r"MI_vs_Time_non_opt.csv", header=None, delimiter='\t')
state = df.to_numpy(dtype=float)
sim_time = state[:,0]
sim_rew = np.zeros((np.shape(sim_time)[0]+1,np.shape(state)[1]-1),dtype=float)
sim_rew[0,:] = nn_fn(data0)
sim_rew[1:,:] = state[:,1:]
mean_rew_ul = avg_rew_20[:,0].reshape(-1,1)

In [ ]:
std=np.zeros((np.shape(sim_time)[0]+1,1),dtype=float)

In [ ]:
std = np.sqrt(np.sum(np.square(sim_rew-mean_rew_ul),axis=1)/(np.shape(sim_rew)[1]-1)).reshape(-1,1)

In [ ]:
y_plus_ul = (mean_rew_ul+std).flatten()
y_minus_ul = (mean_rew_ul-std).flatten()
x = np.arange(0,50001,20)
del sim_time, sim_rew, std

In [ ]:
plt.figure(3)
plt.plot(x,mean_rew_ul,color='black', label=['pi/2_20s_non_trained'])
# plt.plot(time2,y_plus,label=['y_plus'])
# plt.plot(time2,y_minus,label=['y_minus'])
plt.fill_between(x, y_minus_ul, y_plus_ul, color='green', alpha=0.5)#, label='ul_band')

plt.plot(x,mean_rew_l,color='blue', label=['pi/2_20s_trained'])
# plt.plot(time2,y_plus,label=['y_plus'])
# plt.plot(time2,y_minus,label=['y_minus'])
plt.fill_between(x, y_minus_l, y_plus_l, color='grey', alpha=0.5)#, label='l_band')

plt.xlim([0, 50000]) 
plt.ylim([0.0, 1.10])
plt.xlabel('Time')
plt.ylabel('Mix Index')
plt.legend(loc="lower right")
plt.savefig("MI_vs_Time_pi_2_20.png",dpi=1000)
plt.show()
plt.close()